In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'pandas'

## Step 1: 
Import modules and load data

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

home_data_path = 'input/home-data-for-ml-course/train.csv'
home_data = pd.read_csv(home_data_path)

## Step 2
Check out the data

In [3]:
print(home_data.columns)
print(home_data.describe())
print(home_data.head())

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

## Step 3
Get x and y data

In [4]:
my_imputer = SimpleImputer()
features = [col for col in home_data.columns if home_data[col].dtype != 'object' 
           and col != 'SalePrice']
X = home_data[features]
y = home_data['SalePrice']

train_X, val_X, train_y, val_y = train_test_split(X, y)
train_X_imputed = my_imputer.fit_transform(train_X)
val_X_imputed = my_imputer.fit_transform(val_X)
print(train_X.head())

        Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
299    300          20         80.0    14000            6            8   
341    342          20         60.0     8400            4            4   
1119  1120          20         70.0     7560            5            5   
774    775          20        110.0    14226            8            5   
761    762          30         60.0     6911            5            5   

      YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  GarageArea  \
299        1950          2004         0.0           0  ...         300   
341        1950          1950         0.0           0  ...         294   
1119       1959          1959         0.0         369  ...         286   
774        2006          2006       375.0           0  ...         895   
761        1924          1950         0.0         405  ...         440   

      WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  \
299            0           36         

## Step 4
Create Random Forest Model

In [5]:
housing_model = RandomForestRegressor(random_state=1)
housing_model.fit(train_X_imputed, train_y)

RandomForestRegressor(random_state=1)

## Step 5
Predict house prices

In [6]:
house_price_predictions = housing_model.predict(val_X_imputed)
print(mean_absolute_error(house_price_predictions, val_y))

16718.590986301366


## Step 6
Test model with test.csv

In [7]:
test_data_path = 'input/home-data-for-ml-course/test.csv'
X_test = pd.read_csv(test_data_path)
X_test_fixed = X_test[features]
X_test_fixed_imputed = my_imputer.fit_transform(X_test_fixed)
test_predictions = housing_model.predict(X_test_fixed_imputed)

## Step 7
Output the results

In [8]:
# Run the code to save predictions in the format used for competition scoring

output = pd.DataFrame({'Id': X_test.Id,
                       'SalePrice': test_predictions})
output.to_csv('output/submission.csv', index=False)